In [1]:
from face2embeddings.data.setup import create_dataloaders
from pathlib import Path
from face2embeddings.model import FaceSwin
from face2embeddings.utils import create_writer
from face2embeddings.engine import train
from torchvision.transforms import v2
import torch

In [2]:
transform = v2.Compose([
    v2.RandomHorizontalFlip(p=0.3),
    v2.RandomRotation(degrees=(-10, 10), interpolation=v2.InterpolationMode.BILINEAR),
    v2.ColorJitter(brightness=.05, hue=.05, saturation=.05, contrast=.05),
])

In [3]:
train_dataloader, val_dataloader = create_dataloaders(
    train_dir=Path(r"C:\Users\emely\OneDrive\Desktop\face-auth-dataset\train"),
    val_dir=Path(r"C:\Users\emely\OneDrive\Desktop\face-auth-dataset\val"),
    batch_size=5,
    transform=transform,
    num_workers=6,
    train_length=30_000 * 5
)

In [4]:
writer = create_writer(experiment_name="train-13", model_name="FaceSwin", extra="SwinBase")

[INFO] Created SummaryWriter, saving to: runs\2024-06-28\train-13\FaceSwin\SwinBase...


In [5]:
model = FaceSwin(train_from_default=True).to("cuda")
loss_fn = torch.nn.BCELoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=0.01)
scheduler = torch.optim.lr_scheduler.LinearLR(optimizer, start_factor=0.01, end_factor=1, total_iters=1, verbose=True)

C:\Users\emely\OneDrive\Desktop\Papper-face-embedding-model\venv\Lib\site-packages\torch\optim\lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


In [6]:
model.load_state_dict(torch.load("model/train/train-12/checkpoint_2000.pt"))

<All keys matched successfully>

In [7]:
results = train(
    model=model,
    train_dataloader=train_dataloader,
    val_dataloader=val_dataloader,
    loss_fn=loss_fn,
    optimizer=optimizer,
    scheduler=None,
    epochs=1,
    writer=writer,
    device=torch.device("cuda"),
    checkpoint_step=2000,
    checkpoint_path=Path("./model/train/train-13"),
)

Epochs Loop:   0%|          | 0/1 [00:00<?, ?it/s]

Train Step:   0%|          | 0/30001 [00:00<?, ?it/s]

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [9]:
model.to("cpu")

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [8]:
torch.save(model.state_dict(), "./model/train/train-8/model.pt")

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
